In [50]:
# load libraries
import numpy as np
import pyxpcm
from pyxpcm.models import pcm

import xarray as xr

from argopy import DataFetcher as ArgoDataFetcher

import cartopy.crs as ccrs 
import matplotlib.pyplot as plt
import cartopy.feature as cfeature
from datetime import datetime, timedelta
import gsw 

In [92]:
llon=-90;rlon=0
ulat=70;llat=0 
depthmin=0;depthmax=4000
# January 
time_in='2018-01-01'
time_f='2018-04-03'

In [38]:
def get_argo_region_data(llon,rlon,llat,ulat,depthmin,depthmax,time_in,time_f):
    # Function to get argo data for a given lat,lon box (using Argopy), 
    # and return a 2D array collection of vertical profile for the given region
    #llon: left longitude
    #rlon: right longtidue
    #ulat: upper latitude
    #llat: lower latitude

    ds_points = ArgoDataFetcher(src='erddap').region([llon,rlon, llat,ulat, depthmin, depthmax,time_in,time_f]).to_xarray()
    ds_profiles = ds_points.argo.point2profile()
    return ds_profiles

In [ ]:
def spliced_argo_region_data(llon,rlon,llat,ulat,depthmin,depthmax,time_start,time_end):
    #Function that will get the argo data
    #llon: left longitude
    #rlon: right longtidue
    #ulat: upper latitude
    #llat: lower latitude
    #time
    
    max_dt = timedelta(days = 1)
    
    if isinstance(time_start, str):
        time_start = datetime.strptime(time_start,"%Y-%m-%d")
    if isinstance(time_end, str):
        time_end = datetime.strptime(time_end,"%Y-%m-%d")
        
    if time_end - time_start <= max_dt:
        ds = get_argo_region_data(llon,rlon,llat,ulat,depthmin,depthmax,time_start,time_end)
        return ds
    else:
        early_end = time_start+max_dt
        ds = get_argo_region_data(llon,rlon,llat,ulat,depthmin,depthmax,time_start,early_end)
        print("Retrived data from " + time_start + " to " + early_end ", retreived " + len(ds.N_PROF) + " profiles")
        ds2 = spliced_argo_region_data(llon,rlon,llat,ulat,depthmin,depthmax, early_end,time_end)
        return xr.concat([ds1,ds2],dim='N_PROF')


In [ ]:
spliced_argo_region_data(llon,rlon,llat,ulat,depthmin,depthmax,time_in,time_f)